In [1]:
import pandas as pd
import seaborn as sns
from sklearn import preprocessing

## EDA Doença Crônica nos Rins


| **Variável**                  | **Abreviação** | **Tipo**    | **Unidade/Valores**                         |
|-------------------------------|---------------|-------------|---------------------------------------------|
| Age                            | age           | Numérico    | anos                                        |
| Blood Pressure                 | bp            | Numérico    | mm/Hg                                       |
| Specific Gravity               | sg            | Nominal     | (1.005, 1.010, 1.015, 1.020, 1.025)         |
| Albumin                        | al            | Nominal     | (0, 1, 2, 3, 4, 5)                          |
| Sugar                          | su            | Nominal     | (0, 1, 2, 3, 4, 5)                          |
| Red Blood Cells                | rbc           | Nominal     | (normal, abnormal)                          |
| Pus Cell                       | pc            | Nominal     | (normal, abnormal)                          |
| Pus Cell Clumps                | pcc           | Nominal     | (present, notpresent)                       |
| Bacteria                       | ba            | Nominal     | (present, notpresent)                       |
| Blood Glucose Random           | bgr           | Numérico    | mgs/dl                                      |
| Blood Urea                     | bu            | Numérico    | mgs/dl                                      |
| Serum Creatinine               | sc            | Numérico    | mgs/dl                                      |
| Sodium                         | sod           | Numérico    | mEq/L                                       |
| Potassium                      | pot           | Numérico    | mEq/L                                       |
| Hemoglobin                     | hemo          | Numérico    | gms                                         |
| Packed Cell Volume             | pcv           | Numérico    |                                             |
| White Blood Cell Count         | wc            | Numérico    | cells/cumm                                  |
| Red Blood Cell Count           | rc            | Numérico    | millions/cmm                                |
| Hypertension                   | htn           | Nominal     | (yes, no)                                   |
| Diabetes Mellitus              | dm            | Nominal     | (yes, no)                                   |
| Coronary Artery Disease        | cad           | Nominal     | (yes, no)                                   |
| Appetite                       | appet         | Nominal     | (good, poor)                                |
| Pedal Edema                    | pe            | Nominal     | (yes, no)                                   |
| Anemia                         | ane           | Nominal     | (yes, no)                                   |
| Class                          | class         | Nominal     | (ckd, notckd)                               |



1. Preparar os dados
2. Analise descritivia (Medidas de posicao e dispersao,  )
3. Graficos
4. Correlacao
5. Outliers
6. Tratar missing values



[Link para o dataset](https://archive.ics.uci.edu/dataset/336/chronic+kidney+disease)

## 1. Análise Inicial

In [2]:
df = pd.read_csv('data/kidney_disease.csv')
df.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [3]:
#Dimenssão do dataset
print("Número de linhas: ", df.shape[0])
print("Número de colunas: ", df.shape[1])

Número de linhas:  400
Número de colunas:  26


## 2. Preparação dos dados

In [4]:
#Drop coluna de id
df.drop('id',axis=1 ,inplace=True)

In [5]:
#Renomear colunas para um melhor entendimento
df.columns = ['age', 'blood_pressure', 'specific_gravity', 'albumin', 'sugar', 'red_blood_cells', 'pus_cell',
              'pus_cell_clumps', 'bacteria', 'blood_glucose_random', 'blood_urea', 'serum_creatinine', 'sodium',
              'potassium', 'haemoglobin', 'packed_cell_volume', 'white_blood_cell_count', 'red_blood_cell_count',
              'hypertension', 'diabetes_mellitus', 'coronary_artery_disease', 'appetite', 'peda_edema',
              'anemia', 'class']

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      391 non-null    float64
 1   blood_pressure           388 non-null    float64
 2   specific_gravity         353 non-null    float64
 3   albumin                  354 non-null    float64
 4   sugar                    351 non-null    float64
 5   red_blood_cells          248 non-null    object 
 6   pus_cell                 335 non-null    object 
 7   pus_cell_clumps          396 non-null    object 
 8   bacteria                 396 non-null    object 
 9   blood_glucose_random     356 non-null    float64
 10  blood_urea               381 non-null    float64
 11  serum_creatinine         383 non-null    float64
 12  sodium                   313 non-null    float64
 13  potassium                312 non-null    float64
 14  haemoglobin              3

Alguns dados estão representados por tipos que inadequados:
1. age (float) -> (int)
2. sg(float) -> nominal
2. albumin (float) -> nominal (int)
3. sugar (float) -> nominal (int)

4. packed_cell_volume(object) -> (int)
5. white_blood_cell_count (object) -> (int)
6. red_blood_cell_count (object) -> (int)

In [8]:
#Realizar a transformação de tipo dos dados:

df['packed_cell_volume'] = pd.to_numeric(df['packed_cell_volume'], errors='coerce')
df['white_blood_cell_count'] = pd.to_numeric(df['white_blood_cell_count'], errors='coerce')
df['red_blood_cell_count'] = pd.to_numeric(df['red_blood_cell_count'], errors='coerce')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      391 non-null    float64
 1   blood_pressure           388 non-null    float64
 2   specific_gravity         353 non-null    float64
 3   albumin                  391 non-null    float64
 4   sugar                    391 non-null    float64
 5   red_blood_cells          248 non-null    object 
 6   pus_cell                 335 non-null    object 
 7   pus_cell_clumps          396 non-null    object 
 8   bacteria                 396 non-null    object 
 9   blood_glucose_random     356 non-null    float64
 10  blood_urea               381 non-null    float64
 11  serum_creatinine         383 non-null    float64
 12  sodium                   313 non-null    float64
 13  potassium                312 non-null    float64
 14  haemoglobin              3

In [12]:
categ_cols = [col for col in df.columns if df[col].dtype == 'object']
numeric_cols = [col for col in df.columns if df[col].dtype != 'object']

#### Data pre-processing

In [10]:
#Check for missing values
if df.isnull().values.any():
  print("There are missing values in dataset")

There are missing values in dataset


In [24]:
#Encode categorial labels
keys = list(df)
encoder = preprocessing.LabelEncoder()
for key in keys:
  if key != 'AGE':
    df[key] = encoder.fit_transform(df[key])


In [9]:
df.describe()

,age,blood_pressure,specific_gravity,albumin,sugar,blood_glucose_random,blood_urea,serum_creatinine,sodium,potassium,haemoglobin,packed_cell_volume,white_blood_cell_count,red_blood_cell_count
count,391.000000,388.000000,353.000000,391.000000,391.000000,356.000000,381.000000,383.000000,313.000000,312.000000,348.000000,329.000000,294.000000,269.000000
mean,51.483376,76.469072,1.017408,51.483376,51.483376,148.036517,57.425722,3.072454,137.528754,4.627244,12.526437,38.884498,8406.122449,4.707435
std,17.169714,13.683637,0.005717,17.169714,17.169714,79.281714,50.503006,5.741126,10.408752,3.193904,2.912587,8.990105,2944.474190,1.025323
min,2.000000,50.000000,1.005000,2.000000,2.000000,22.000000,1.500000,0.400000,4.500000,2.500000,3.100000,9.000000,2200.000000,2.100000
25%,42.000000,70.000000,1.010000,42.000000,42.000000,99.000000,27.000000,0.900000,135.000000,3.800000,10.300000,32.000000,6500.000000,3.900000
50%,55.000000,80.000000,1.020000,55.000000,55.000000,121.000000,42.000000,1.300000,138.000000,4.400000,12.650000,40.000000,8000.000000,4.800000
75%,64.500000,80.000000,1.020000,64.500000,64.500000,163.000000,66.000000,2.800000,142.000000,4.900000,15.000000,45.000000,9800.000000,5.400000
max,90.000000,180.000000,1.025000,90.000000,90.000000,490.000000,391.000000,76.000000,163.000000,47.000000,17.800000,54.000000,26400.000000,8.000000
